In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import re

In [2]:
output = dd.read_csv('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/output_*.csv')
evolved = dd.read_table('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/evolved_*.dat', sep='\s+')
logfile = dd.read_csv('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/logfile_*.dat', header =None).squeeze()
# logfile = dd.read_table('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/logfile_*.dat', sep=';', header=None, names=['Object', 'name', 'ID', 'event', 'time', 'info'])

print(f'Number of pratitions for evolved files : {evolved.npartitions}')
print(f'Number of pratitions for output files : {output.npartitions}')
print(f'Number of pratitions for logfile files : {logfile.npartitions}')

Number of pratitions for evolved files : 30
Number of pratitions for output files : 60
Number of pratitions for logfile files : 30


Filtering logfile files to count the number of RL overflows and CE

In [34]:
a = logfile.str.findall(r"B;(?:\d*\_)?\d+;(\d+);RLO_BEGIN;").compute()

In [35]:
a.head(10)

0     []
1     []
2     []
3     []
4     []
5    [1]
6     []
7     []
8     []
9     []
Name: 0, dtype: object

In [73]:
b = a.to_frame(name='ID').explode('ID').groupby('ID').size().to_frame(name='RLO').reset_index()

In [76]:
b[b['ID']==6]

,ID,RLO


Filtering output files to get only binaries black holes

In [ ]:
idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
output_bhbh=output[idxBHBH] #Distributed DataFrame containing only binaries BB
# output_bhbh = output_bhbh.compute()